In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seq2seq-predictions/final_seq2seq_generations.csv


In [2]:
pd.set_option('display.max_colwidth', None)

### Analysis for seq2seq predictions

In [25]:
df_seq2seq = pd.read_csv("/kaggle/input/seq2seq-predictions/final_seq2seq_generations.csv")
df_seq2seq.head()

,text,headlines,predictions
0,the season premiere of hbo game of thrones became the most watched in the show history with 10.1 million viewers tuning in to the live broadcast as per hbo this season first episode beat viewership for last year season finale which had 8.89 million viewers it was also the most tweeted got episode ever with 2.4 million tweets.,season premiere most watched got episode in showã¢ââs history,start game of thrones actor kit gets 10 billion from film end
1,president pranab mukherjee is entitled to salary of ã¢ââ¹75 000 per month half of his salary as president after he retires on july 24 he will further be entitled to office expenses of ã¢ââ¹60 000 per annum mobile phone car and free travel across india he will live in the delhi bungalow where former president dr apj abdul kalam stayed.,what will president pranab mukherjee earn post retirement,start prez kovind announces pay for lakh over lakh people end
2,singer ed sheeran deleted his twitter account on monday after announcing earlier this month that he taking break from the social media platform go on it and there nothing but people saying mean things he had said earlier notably the singer was trolled for his cameo appearance on sunday premiere episode of game of thrones season 7.,singer ed sheeran deletes his twitter account,start twitter users react to twitter account hacked end
3,nawazuddin siddiqui while speaking about his tweet about being discriminated in the film industry on the basis of his skin colour said it was just reply to someone in the film industry nawazuddin further explained that his tweet was not directed towards the entire industry these things are there everywhere not just in bollywood he added.,was reply to someone siddiqui on tweet about skin colour,start nawazuddin is the name of the film nawazuddin end
4,twitter has deleted nearly 90 000 fake accounts after security researchers discovered massive pornographic botnet that was posting links to scam websites and offering online sex the fake profiles had pictures of females and each tweet included sexually explicit phrase prompting users to click on the accompanying url the accounts were reportedly able to attract over crore clicks.,twitter deletes 90 000 fake accounts that offered online sex,start facebook faces global outage for fake news end


In [4]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=9e46abc0e8e1065485af15b1c3dcb566aad613e819a6591cf5b7025ac66e2fc0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [26]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(df_seq2seq['predictions'],df_seq2seq['headlines']):
    scores = scorer.score(pred, ref)
    rouge_scores.append(scores)

# Extract ROUGE scores
rouge_1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge_2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rouge_l_scores = [score['rougeL'].fmeasure for score in rouge_scores]

# Add ROUGE scores as new columns in the original DataFrame
df_seq2seq['Rouge_R1'] = rouge_1_scores
df_seq2seq['Rouge_R2'] = rouge_2_scores
df_seq2seq['Rouge_RL'] = rouge_l_scores

In [28]:
df_seq2seq['predictions'] = df_seq2seq['predictions'].str.replace('start ', '')
df_seq2seq['predictions'] = df_seq2seq['predictions'].str.replace(' end', '')

In [29]:
top_five_indexes = df_seq2seq.nlargest(5, 'Rouge_R1').index

print("Indexes of top five rows with highest rouge-1 scores:")
print(top_five_indexes)

Indexes of top five rows with highest rouge-1 scores:
Index([99, 20, 19, 8, 36], dtype='int64')


In [30]:
df_seq2seq.iloc[top_five_indexes]

,text,headlines,predictions,Rouge_R1,Rouge_R2,Rouge_RL
99,three terrorists have been killed in an ongoing encounter late on monday in jammu and kashmir anantnag area according to reports the encounter was carried out by joint team of security forces and the identities of the terrorists are being ascertained police officials said the police have also recovered weapons from the terrorists reports added,3 terrorists killed in encounter in anantnag,2 terrorists killed in encounter in k,0.625000,0.571429,0.625000
20,a priest from kerala wayanad district identified as father saji joseph has been arrested for sexually abusing minor boys at children home the incident that took place during the summer of 2016 came to light recently after victim shared the ordeal with his parents the accused was absconding for the past three days before being arrested from karnataka.,kerala priest arrested for sexually abusing minor boys,kerala school teacher arrested for raping minor boys,0.555556,0.250000,0.555556
19,the centre has asked all the states to compulsorily file firs in all untoward incidents happening in the name of cow protection the lok sabha was informed on tuesday the centre said that the responsibility to maintain law and order and preventing incidents of attacks on cattle traders beef eaters muslims dalits and dairy farmers rests with the states.,govt asks states to file firs over violence in cow name,govt asks states to stop using cow protection from centre,0.434783,0.285714,0.434783
8,hackers on monday stole over ã¢ââ¹45 crore worth of digital currency ether in about three minutes by tricking victims into sending money to the wrong web address the hackers changed the address which was posted by company called coindash to get funds from investors it took coindash three minutes to realise its investors were sending funds to hackers address.,hackers steal ã¢ââ¹45 crore-worth digital currency in minutes,hackers steal million worth ã¢ââ¹1 crore from bitcoin,0.421053,0.117647,0.315789
36,a video captured by passenger commuting on bus shows the driver peeling an apple while driving on highway in taizhou china the driver can be seen holding the steering and calmly peeling an apple and tossing the discarded peels in bucket at the same time he has reportedly been banned from driving passenger vehicles for life.,video shows man peeling an apple while driving bus,video shows apple watch on self driving car in china,0.380952,0.105263,0.380952


In [31]:
import nltk.translate.bleu_score as bleu
import nltk

In [32]:
# Function to calculate BLEU score for a pair of original and predicted texts
def calculate_bleu_score(original_text, predicted_text):
    
    # Tokenize the texts
    reference = nltk.word_tokenize(original_text.lower())
    candidate = nltk.word_tokenize(predicted_text.lower())

    # Calculate BLEU score
    smoother = bleu.SmoothingFunction()
#     bleu_score = bleu.sentence_bleu([reference], candidate, smoothing_function=smoother.method1)

    bleu_score = bleu.sentence_bleu([reference], candidate)

    return bleu_score

# Apply the function to each row in the DataFrame
df_seq2seq['BLEU'] = df_seq2seq.apply(lambda row: calculate_bleu_score(row['headlines'], row['predictions']), axis=1)

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [24]:
df_seq2seq.iloc[top_five_indexes]

,text,headlines,predictions,Rouge_R1,Rouge_R2,Rouge_RL,BLEU
99,three terrorists have been killed in an ongoing encounter late on monday in jammu and kashmir anantnag area according to reports the encounter was carried out by joint team of security forces and the identities of the terrorists are being ascertained police officials said the police have also recovered weapons from the terrorists reports added,3 terrorists killed in encounter in anantnag,2 terrorists killed in encounter in k,0.625000,0.571429,0.625000,0.614788
20,a priest from kerala wayanad district identified as father saji joseph has been arrested for sexually abusing minor boys at children home the incident that took place during the summer of 2016 came to light recently after victim shared the ordeal with his parents the accused was absconding for the past three days before being arrested from karnataka.,kerala priest arrested for sexually abusing minor boys,kerala school teacher arrested for raping minor boys,0.555556,0.250000,0.555556,0.650059
19,the centre has asked all the states to compulsorily file firs in all untoward incidents happening in the name of cow protection the lok sabha was informed on tuesday the centre said that the responsibility to maintain law and order and preventing incidents of attacks on cattle traders beef eaters muslims dalits and dairy farmers rests with the states.,govt asks states to file firs over violence in cow name,govt asks states to stop using cow protection from centre,0.434783,0.285714,0.434783,0.251329
8,hackers on monday stole over ã¢ââ¹45 crore worth of digital currency ether in about three minutes by tricking victims into sending money to the wrong web address the hackers changed the address which was posted by company called coindash to get funds from investors it took coindash three minutes to realise its investors were sending funds to hackers address.,hackers steal ã¢ââ¹45 crore-worth digital currency in minutes,hackers steal million worth ã¢ââ¹1 crore from bitcoin,0.421053,0.117647,0.315789,0.434721
36,a video captured by passenger commuting on bus shows the driver peeling an apple while driving on highway in taizhou china the driver can be seen holding the steering and calmly peeling an apple and tossing the discarded peels in bucket at the same time he has reportedly been banned from driving passenger vehicles for life.,video shows man peeling an apple while driving bus,video shows apple watch on self driving car in china,0.380952,0.105263,0.380952,0.459150


### Analysis for Bert Predictions

In [34]:
df_bert = pd.read_csv('/kaggle/input/bert-predictions/final_bert_generations.csv')
df_bert.drop(columns = ['Truncated_headlines'], inplace=True)
df_bert

,text,headlines,Predicted_headline
0,the season premiere of hbo game of thrones became the most watched in the show history with 10.1 million viewers tuning in to the live broadcast as per hbo this season first episode beat viewership for last year season finale which had 8.89 million viewers it was also the most tweeted got episode ever with 2.4 million tweets.,season premiere most watched got episode in showa¢aas history,the season premiere of hbo game of thrones became the most watched in the show history
1,president pranab mukherjee is entitled to salary of a¢aa¹75 000 per month half of his salary as president after he retires on july 24 he will further be entitled to office expenses of a¢aa¹60 000 per annum mobile phone car and free travel across india he will live in the delhi bungalow where former president dr apj abdul kalam stayed.,what will president pranab mukherjee earn post retirement,president pranab mukherjee will live in the delhi bungalow where former president dr apj abdul kalam stayed<q>he will also be entitled to office expenses of [UNK] 000 per month
2,singer ed sheeran deleted his twitter account on monday after announcing earlier this month that he taking break from the social media platform go on it and there nothing but people saying mean things he had said earlier notably the singer was trolled for his cameo appearance on sunday premiere episode of game of thrones season 7.,singer ed sheeran deletes his twitter account,ed sheeran deleted his twitter account on monday after announcing that he taking break from the social media platform go on it
3,nawazuddin siddiqui while speaking about his tweet about being discriminated in the film industry on the basis of his skin colour said it was just reply to someone in the film industry nawazuddin further explained that his tweet was not directed towards the entire industry these things are there everywhere not just in bollywood he added.,was reply to someone siddiqui on tweet about skin colour,nawazuddin siddiqui spoke about being discriminated in the film industry on the basis of his skin colour
4,twitter has deleted nearly 90 000 fake accounts after security researchers discovered massive pornographic botnet that was posting links to scam websites and offering online sex the fake profiles had pictures of females and each tweet included sexually explicit phrase prompting users to click on the accompanying url the accounts were reportedly able to attract over crore clicks.,twitter deletes 90 000 fake accounts that offered online sex,security researchers discovered massive pornographic botnet that was posting links to scam websites and offering online sex<q>the accounts were reportedly able to attract over thousands of clicks
...,...,...,...
995,sonam kapoor is set to join the harvard business school hbs panel for special talk think the subject is very relevant to times that we live in said sonam she will reportedly talk about the pitfalls and challenges associated with both the aspects notions of beauty and the aspiration to look beautiful as per source,sonam to be part of harvard business school panel for talk,sonam is set to join the harvard business school panel for special talk<q>she will reportedly talk about the pitfalls and challenges associated with beauty and the aspiration to look beautiful as per per per person
996,rss chief mohan bhagwat on wednesday said our society has found an able and competent contractor pm narendra modi for social welfare he added but the danger is that we might put the entire onus on the contractor head and sleep this shouldn happen the people who work for public welfare have feeling of worship for the country,modi is contractor of social welfare rss chief bhagwat,rss chief mohan bhagwat says our society has found an able and competent contractor for social welfare
997,the record for the highest last-wicket partnership in test cricket was set on july 12 by england james anderson and joe root while facing india in 2014 th

In [36]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(df_bert['Predicted_headline'],df_bert['headlines']):
    scores = scorer.score(pred, ref)
    rouge_scores.append(scores)

# Extract ROUGE scores
rouge_1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge_2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rouge_l_scores = [score['rougeL'].fmeasure for score in rouge_scores]

# Add ROUGE scores as new columns in the original DataFrame
df_bert['Rouge_R1'] = rouge_1_scores
df_bert['Rouge_R2'] = rouge_2_scores
df_bert['Rouge_RL'] = rouge_l_scores

In [37]:
df_bert.head()

,text,headlines,Predicted_headline,Rouge_R1,Rouge_R2,Rouge_RL
0,the season premiere of hbo game of thrones became the most watched in the show history with 10.1 million viewers tuning in to the live broadcast as per hbo this season first episode beat viewership for last year season finale which had 8.89 million viewers it was also the most tweeted got episode ever with 2.4 million tweets.,season premiere most watched got episode in showa¢aas history,the season premiere of hbo game of thrones became the most watched in the show history,0.428571,0.153846,0.428571
1,president pranab mukherjee is entitled to salary of a¢aa¹75 000 per month half of his salary as president after he retires on july 24 he will further be entitled to office expenses of a¢aa¹60 000 per annum mobile phone car and free travel across india he will live in the delhi bungalow where former president dr apj abdul kalam stayed.,what will president pranab mukherjee earn post retirement,president pranab mukherjee will live in the delhi bungalow where former president dr apj abdul kalam stayed<q>he will also be entitled to office expenses of [UNK] 000 per month,0.205128,0.108108,0.153846
2,singer ed sheeran deleted his twitter account on monday after announcing earlier this month that he taking break from the social media platform go on it and there nothing but people saying mean things he had said earlier notably the singer was trolled for his cameo appearance on sunday premiere episode of game of thrones season 7.,singer ed sheeran deletes his twitter account,ed sheeran deleted his twitter account on monday after announcing that he taking break from the social media platform go on it,0.413793,0.370370,0.413793
3,nawazuddin siddiqui while speaking about his tweet about being discriminated in the film industry on the basis of his skin colour said it was just reply to someone in the film industry nawazuddin further explained that his tweet was not directed towards the entire industry these things are there everywhere not just in bollywood he added.,was reply to someone siddiqui on tweet about skin colour,nawazuddin siddiqui spoke about being discriminated in the film industry on the basis of his skin colour,0.370370,0.080000,0.296296
4,twitter has deleted nearly 90 000 fake accounts after security researchers discovered massive pornographic botnet that was posting links to scam websites and offering online sex the fake profiles had pictures of females and each tweet included sexually explicit phrase prompting users to click on the accompanying url the accounts were reportedly able to attract over crore clicks.,twitter deletes 90 000 fake accounts that offered online sex,security researchers discovered massive pornographic botnet that was posting links to scam websites and offering online sex<q>the accounts were reportedly able to attract over thousands of clicks,0.256410,0.108108,0.205128


In [38]:
# Function to calculate BLEU score for a pair of original and predicted texts
def calculate_bleu_score(original_text, predicted_text):
    
    # Tokenize the texts
    reference = nltk.word_tokenize(original_text.lower())
    candidate = nltk.word_tokenize(predicted_text.lower())

    # Calculate BLEU score
    smoother = bleu.SmoothingFunction()
#     bleu_score = bleu.sentence_bleu([reference], candidate, smoothing_function=smoother.method1)

    bleu_score = bleu.sentence_bleu([reference], candidate)

    return bleu_score

# Apply the function to each row in the DataFrame
df_bert['BLEU'] = df_bert.apply(lambda row: calculate_bleu_score(row['headlines'], row['Predicted_headline']), axis=1)

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [39]:
df_bert.iloc[top_five_indexes]

,text,headlines,Predicted_headline,Rouge_R1,Rouge_R2,Rouge_RL,BLEU
99,three terrorists have been killed in an ongoing encounter late on monday in jammu and kashmir anantnag area according to reports the encounter was carried out by joint team of security forces and the identities of the terrorists are being ascertained police officials said the police have also recovered weapons from the terrorists reports added,3 terrorists killed in encounter in anantnag,three terrorists have been killed in an ongoing encounter in the jammu and kashmir anantnag area<q>police have also recovered weapons from the terrorists,0.375000,0.133333,0.375000,0.361586
20,a priest from kerala wayanad district identified as father saji joseph has been arrested for sexually abusing minor boys at children home the incident that took place during the summer of 2016 came to light recently after victim shared the ordeal with his parents the accused was absconding for the past three days before being arrested from karnataka.,kerala priest arrested for sexually abusing minor boys,a priest from kerala wayanad district has been arrested for sexually abusing minor boys at children home<q>saji joseph was absconding for the past three days before being arrested from karnataka wayanad,0.390244,0.256410,0.341463,0.139800
19,the centre has asked all the states to compulsorily file firs in all untoward incidents happening in the name of cow protection the lok sabha was informed on tuesday the centre said that the responsibility to maintain law and order and preventing incidents of attacks on cattle traders beef eaters muslims dalits and dairy farmers rests with the states.,govt asks states to file firs over violence in cow name,the centre has asked all the states to compulsorily file firs in all untoward incidents happening in the name of cow protection,0.484848,0.129032,0.424242,0.417226
8,hackers on monday stole over a¢aa¹45 crore worth of digital currency ether in about three minutes by tricking victims into sending money to the wrong web address the hackers changed the address which was posted by company called coindash to get funds from investors it took coindash three minutes to realise its investors were sending funds to hackers address.,hackers steal a¢aa¹45 crore-worth digital currency in minutes,the hackers changed the address posted by company coindash to get funds from investors,0.076923,0.000000,0.076923,0.516973
36,a video captured by passenger commuting on bus shows the driver peeling an apple while driving on highway in taizhou china the driver can be seen holding the steering and calmly peeling an apple and tossing the discarded peels in bucket at the same time he has reportedly been banned from driving passenger vehicles for life.,video shows man peeling an apple while driving bus,the driver has reportedly been banned from driving passenger vehicles for life<q>he can be seen casually peeling an apple and throwing the peels in bucket,0.222222,0.117647,0.166667,0.138211
